In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scipy.io

from config import Config
import utils
import model as modellib
import visualize
from model import log
import prostate

%matplotlib inline

# GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


In [18]:
dataset_dir = "/scratch/wenyuan/Mask_RCNN_On_Pathology/Data_Pre_Processing/cedars-224"
held_out_set = 4
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set) # take out the mean pixel for specific held_out_set

class InferenceConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    DETECTION_NMS_THRESHOLD = 1     
    MEAN_PIXEL = np.array(mean_pixel)

inference_config = InferenceConfig()
inference_config.display()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)
# Get path to saved weights
# Either set a specific path, find a trained weights specified by epoch and held_out_set or find last trained weights

# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
# model_path = model.find_last()[1]
# model_path = model.find_specific(epoch = 80, held_out_set = held_out_set)[1]
model_path = model.find_specific(epoch = 70)[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)


Configurations:
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        1
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 193.97800579  120.89113632  183.79060979]
MINI_MASK_SHAPE                (56, 56)
NAME                           prostate
NUM_CLASSES                    4
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST

In [19]:
def instance_2_sementic(instance_mask, class_ids):
    """convert instance mask to sementic mask
    """
    try:
        h, w, d = instance_mask.shape
    except ValueError:
        mask = int(class_ids) * instance_mask 
        result_dict = {'ATmask': mask}
        return result_dict
    
    mask_map = {}
    for index, label in enumerate(class_ids):
        mask_map[str(label)] = np.logical_or(mask_map[str(label)], \
                                             instance_mask[:, :, index]) \
        if str(label) in mask_map.keys() else instance_mask[:, :, index]
    
    mask = np.zeros((h, w), dtype=np.int)

    for key in mask_map.keys():
        if (key != '0'):
            mask = mask + int(key) * mask_map[key] 
    result_dict = {'ATmask': mask}
    return result_dict

In [20]:
def prediction_2_sementic(mask, class_ids, scores):
    """ convert maskrcnn prediction to 1 single sementic mask
    """
    mask_map = {} # create a probability dict for each class
    for index, label in enumerate(class_ids):
        mask_map[str(label)] = \
        np.maximum(mask_map[str(label)], scores[index] * mask[:, :, index])\
        if str(label) in mask_map.keys() else scores[index] * mask[:, :, index]
    
    ## convert to h * w * num_classes probability map
    h, w, d = mask.shape
    for i in range(4):
        try:
            sementic_mask = \
            np.concatenate((sementic_mask, \
                            np.expand_dims(mask_map[str(i)], axis = -1)),axis = -1)\
            if (i != 0) else np.expand_dims(mask_map[str(i)], axis = -1)
        except KeyError:
            sementic_mask = np.concatenate((sementic_mask, np.zeros((h, w, 1))),axis = -1)\
            if (i != 0) else np.zeros((h, w, 1))
            
    sementic_mask_res = np.argmax(sementic_mask, axis = -1)
    return sementic_mask_res

In [21]:
def save_sementic(result_dict, image_id, dataset_dir):
    save_name = \
    os.path.join(dataset_dir, 'masks_sementic_mod/' + str(image_id).zfill(4) + '_sementic.mat')
    scipy.io.savemat(save_name, result_dict)

In [22]:
def compute_mIOU(c_matrix):
    """ compute the mIOU based on the confusion matrix
    """
    num_class, _ = c_matrix.shape
    IOU = []
    for i in range(num_class):
        p = c_matrix[i, i] / (sum(c_matrix[i, :]) + sum(c_matrix[:, i]) - c_matrix[i, i]) \
        if (sum(c_matrix[i, :]) + sum(c_matrix[:, i]) - c_matrix[i, i]) else 0
        IOU.append(p)
    return np.mean(IOU), IOU

In [23]:
dataset_val = prostate.ProstateDataset()
_, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)
# val_list = [0, 1, 2, 3]
# exclude_list = [394, 185, 506, 289, 360, 7, 222, 406,\
#                152, 195, 219, 510, 196, 399, 237, 41,\
#                44, 274, 423, 432, 207, 371, 463, 482,\
#                176, 50, 494, 154, 337, 275, 166, 184,\
#                331, 262, 496, 260, 5, 240, 80, 426, 277,\
#                124, 162, 288, 251, 465, 72, 187, 284, \
#                342, 115, 139, 40, 503, 77, 350, 430, 12, \
#                499, 459, 312, 142, 100, 128]
# val_list = [image for image in val_list if image not in exclude_list]
dataset_val.load_prostate(dataset_dir, val_list, mode = 16)
dataset_val.prepare()

In [24]:
### Define the crop position: the output of the prediction is [512 * 512] while we take [256 * 256] image
height_crop = [128, 384]
width_crop = [128, 384]
save = False # whether to save the gt-sementic mask at size of [512 * 512]
C_MATRIX = np.zeros((4, 4))
for image_id in dataset_val.image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, inference_config,
                                   image_id, use_mini_mask=False)
    # Convert gt-instance mask to gt-sementic mask
    gt_sementic_mask = instance_2_sementic(gt_mask, gt_class_id)
    
    if save:
        # TODO: save the gt-sementic mask at size of [256 * 256]
        print('save file!')
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Convert prediction to sementic mask
    result_sementic = prediction_2_sementic(r['masks'], r['class_ids'], r['scores'])\
    if r['class_ids'].size != 0 else np.zeros((image.shape[0], image.shape[1]))
    # Compute confusion matrix
    c_matrix = confusion_matrix(np.ravel(\
                                gt_sementic_mask['ATmask'][height_crop[0] : height_crop[1], \
                                                           width_crop[0] : width_crop[1]]), \
                                np.ravel(result_sementic[height_crop[0] : height_crop[1], \
                                                         width_crop[0] : width_crop[1]]))
    
    if c_matrix.size != 16:
        """if the confusion matrix is not 4 by 4, expand the matrix to 4 * 4
        """
        unique_set = \
        np.union1d(np.unique(result_sementic[height_crop[0] : height_crop[1], width_crop[0] : width_crop[1]]), \
                   np.unique(gt_sementic_mask['ATmask'][height_crop[0] : height_crop[1], \
                                                        width_crop[0] : width_crop[1]]))
        xv, yv = np.meshgrid(unique_set, unique_set)
        temp = np.zeros((4, 4))
        temp[yv.astype(int), xv.astype(int)] = c_matrix
        c_matrix = temp 
    
    # Update cofusion matrix
    C_MATRIX = C_MATRIX + c_matrix
    if ((image_id + 1) % (16 * 5) == 0):
        print('Done evaluating %d / %d!'%(image_id + 1, len(dataset_val.image_ids)))
        
# Compute mIOU based on confusion matrix
mIOU, IOU = compute_mIOU(C_MATRIX)
print("mIOU: ", mIOU, "\nIOU:", IOU)

Done evaluating 80 / 1648!
Done evaluating 160 / 1648!
Done evaluating 240 / 1648!
Done evaluating 320 / 1648!
Done evaluating 400 / 1648!
Done evaluating 480 / 1648!
Done evaluating 560 / 1648!
Done evaluating 640 / 1648!
Done evaluating 720 / 1648!
Done evaluating 800 / 1648!
Done evaluating 880 / 1648!
Done evaluating 960 / 1648!
Done evaluating 1040 / 1648!
Done evaluating 1120 / 1648!
Done evaluating 1200 / 1648!
Done evaluating 1280 / 1648!
Done evaluating 1360 / 1648!
Done evaluating 1440 / 1648!
Done evaluating 1520 / 1648!
Done evaluating 1600 / 1648!
mIOU:  0.682612452359 
IOU: [0.7024491890225385, 0.65283656483892538, 0.69198332023366893, 0.68318073534275736]


In [25]:
print(C_MATRIX)
mIOU, IOU = compute_mIOU(C_MATRIX)
print("mIOU: ", mIOU, "\nIOU:", IOU)

[[ 38901955.   1836547.   8672610.   2166310.]
 [  1010767.  11031535.   1807173.    313033.]
 [  2516889.    628617.  31208719.     84748.]
 [   275376.    270179.    181636.   7097234.]]
mIOU:  0.682612452359 
IOU: [0.7024491890225385, 0.65283656483892538, 0.69198332023366893, 0.68318073534275736]


In [26]:
# visualize.display_instances(image, gt_bbox, gt_mask, gt_class_id, 
#                              dataset_val.class_names, figsize=(8, 8))

In [27]:
# results = model.detect([image], verbose=0)
# r = results[0]
# visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset_val.class_names, r['scores'], figsize=(8, 8))

In [28]:
# gt_sementic_mask = instance_2_sementic(gt_mask, gt_class_id)

In [29]:
# Visualization the sementic map
# visualize.display_sementic(image, gt_sementic_mask['ATmask'],
#                             figsize=(8, 8))

In [30]:
# # Convert prediction to sementic mask
# result_sementic = prediction_2_sementic(r['masks'], r['class_ids'], r['scores'])\
# if r['class_ids'].size != 0 else np.zeros((image.shape[0], image.shape[1]))

# visualize.display_sementic(image, result_sementic,
#                             figsize=(8, 8))
# # Compute confusion matrix
# c_matrix = confusion_matrix(np.ravel(gt_sementic_mask['ATmask'][50:150, 50:150]), \
#                             np.ravel(result_sementic[50:150, 50:150]))
# if c_matrix.size != 16:
#     """if the confusion matrix is not 4 by 4
#     """
#     unique_set = \
#     np.union1d(np.unique(result_sementic[50:150, 50:150]), \
#                np.unique(gt_sementic_mask['ATmask'][50:150, 50:150]))
#     xv, yv = np.meshgrid(unique_set, unique_set)
#     temp = np.zeros((4, 4))
#     temp[yv.astype(int), xv.astype(int)] = c_matrix
#     c_matrix = temp 

In [31]:
# mIOU, IOU = compute_mIOU(c_matrix)
# print(mIOU, IOU)